**Unit Commitment Seminar**

**Power Markets and Regulations**

**TA: Sahar Moghimian**

# Unit Commitment

_**[Power Systems Optimization](https://github.com/east-winds/power-systems-optimization)**_

_by Michael R. Davidson and Jesse D. Jenkins (last updated: October 11, 2022)_

This notebook will build on the economic dispatch (ED) model by introducing binary startup (called "commitment") decisions, constraints, and costs of thermal generators. Similar to ED, it minimizes the short-run production costs of meeting electricity demand, but these additions are necessary for operating systems with large amounts of (inflexible) thermal generation. They also entail significant computational trade-offs.

****

We build up the model in several stages. We first start with the simplest model that incorporates unit commitment decisions in order to understand the logic of three-variable commitment formulations.

Next, we introduce moderate complexity through the addition of ramp constraints&mdash;which we need to be modified from the ED to account for startups and shutdowns.

Finally, we model a more realistic unit commitment that includes reserves&mdash;an important feature of day-ahead scheduling in power systems.

## Introduction to unit commitment

Engineering considerations severely limit the possible output ranges of power plants. System operators need to be aware of these limitations when scheduling generation to meet demand. Thermal power plants, in particular, due to their complex designs, thermodynamic cycles, and material properties, can be particularly challenging. In practice, due to the times involved in bringing these power plants online, much of this scheduling is done day-ahead, which gives rise to the need for a day-ahead market.

**Unit commitment** (UC) is the problem of minimizing short-run costs of production inclusive of production and startup costs, in order to meet a given demand and considering relevant engineering constraints of the generators. It is built on the ED model in that it typically considers all of the same variables in addition to the new startup-related variables and constraints&mdash;hence, it is also sometimes called unit commitment and economic dispatch (UCED). This notebook, for simplicity, only considers a single-bus case, while typical formulations usually include a simplified network representation.

## Simple unit commitment

We start with the simplest case that incorporates unit commitment, building on the `economic_dispatch_multi` problem of the [Economic Dispatch notebook](04-Economic-Dispatch.ipynb).


$$
\begin{align}
\min \ & \sum_{g \in G, t \in T} VarCost_g \times GEN_{g,t} + \sum_{g \in G_{thermal}, t \in T} StartUpCost_g \times START_{g,t} \\
\end{align}
$$
$$
\begin{align}
\text{s.t.} & \\
 & \sum_{g} GEN_{g,t} = Demand_t & \forall \quad t \in T \\
 & GEN_{g,t} \leq Pmax_{g,t} & \forall \quad g \notin G_{thermal} , t \in T \\
 & GEN_{g,t} \geq Pmin_{g,t} & \forall \quad g \notin G_{thermal} , t \in T \\
 & GEN_{g,t} \leq Pmax_{g,t} \times COMMIT_{g,t} & \forall \quad g \in G_{thermal} , t \in T \\
 & GEN_{g,t} \geq Pmin_{g,t} \times COMMIT_{g,t} & \forall \quad g \in G_{thermal} , t \in T \\
 & COMMIT_{g,t} \geq \sum_{t'\geq t-MinUp_g}^{t} START_{g,t} & \forall \quad g \in G_{thermal} , t \in T \\
 & 1-COMMIT_{g,t} \geq \sum_{t'\geq t-MinDown_g}^{t} SHUT_{g,t} &\forall \quad g \in G_{thermal} , t \in T \\
  & COMMIT_{g,t+1} - COMMIT_{g,t} =&\\
 & \quad START_{g,t+1} - SHUT_{g,t+1} &\forall \quad G_{thermal} \in G , t = 1..T-1
\end{align}
$$


The **decision variables** in the above problem:

- $GEN_{g}$, generation (in MW) produced by each generator, $g$
- $START_{g,t}$, startup decision (binary) of thermal generator $g$ at time $t$
- $SHUT_{g,t}$, shutdown decision (binary) of thermal generator $g$ at time $t$
- $COMMIT_{g,t}$, commitment status (binary) of generator $g$ at time $t$

The **parameters** are:

- $Pmin_g$, the minimum operating bounds for generator $g$ (based on engineering or natural resource constraints)
- $Pmax_g$, the maximum operating bounds for generator $g$ (based on engineering or natural resource constraints)
- $Demand$, the demand (in MW)
- $VarCost_g = VarOM_g + HeatRate_g \times FuelCost_g$, the variable cost of generator $g$
- $StartUpCost_g$, the startup cost of generator $g$
- $MinUp_g$, the minimum up time of generator $g$, or the minimum time after start-up before a unit can shut down
- $MinDown_g$, the minimum down time of generator $g$, or the minimum time after shut-down before a unit can start again

In addition, we introduce a few different sets:

- $G$, the set of all generators
- $G_{thermal} \subset G$, the subset of thermal generators for which commitment is necessary
- $T$, the set of all time periods over which we are optimizing commitment and dispatch decisions

Finally, the **three-variable commitment equations** capture the basic logic of commitment:

- Units incur costs when they startup (not when they shutdown)
- Units must stay on (and off) for a minimum period of time&mdash;in lieu of explicitly enforcing a startup trajectory
- Some summations (simplified here) will need to be modified near the beginning of the time period

There are some further resources at the bottom for alternative and/or more complex formulations of the UC.

Now, let's implement UC.

### 0. Install and import packages

In [ ]:
# Install packages (uncomment if running in Google Colab or fresh environment)
# !pip install pyomo highspy plotly pandas

In [ ]:
import pyomo.environ as pyo
import pandas as pd
import numpy as np
import plotly.express as px

pd.set_option('display.max_columns', 120)
pd.set_option('display.max_rows', 100)

### 1. Load and format data

We will use data loosely based on San Diego Gas and Electric (SDG&E, via the [PowerGenome](https://github.com/gschivley/PowerGenome) data platform) including a few neighboring generators and adjustments to make the problem easy to solve, consisting of:

- 33 generators (we added a few more to ensure we can provide enough reserves)
- estimated hourly demand for 2020 (net load at the transmission substation level after subtracting 600MW of behind-the-meter solar from original demand)
- variable generation capacity factors
- estimated natural gas fuel costs

In order to demonstrate the impacts of unit commitment, we will keep our high solar sensitivity case from the [Economic Dispatch notebook](04-Economic-Dispatch.ipynb) (with 3,500 MW of solar PV) to produce large variations in net load (demand less available renewable supply) required to meet demand (similar to [California's infamous "Duck Curve"](https://www.caiso.com/Documents/FlexibleResourcesHelpRenewables_FastFacts.pdf)).

In [ ]:
# Load data from GitHub
gen_info = pd.read_csv("https://raw.githubusercontent.com/Power-Systems-Optimization-Course/power-systems-optimization/master/Notebooks/uc_data/Generators_data.csv")
fuels = pd.read_csv("https://raw.githubusercontent.com/Power-Systems-Optimization-Course/power-systems-optimization/master/Notebooks/uc_data/Fuels_data.csv")
loads = pd.read_csv("https://raw.githubusercontent.com/Power-Systems-Optimization-Course/power-systems-optimization/master/Notebooks/uc_data/Demand.csv")
gen_variable = pd.read_csv("https://raw.githubusercontent.com/Power-Systems-Optimization-Course/power-systems-optimization/master/Notebooks/uc_data/Generators_variability.csv")

# Rename all columns to lowercase
for df in [gen_info, fuels, loads, gen_variable]:
    df.columns = [c.lower() for c in df.columns]

**Construct generator dataframe**

In [ ]:
# Keep columns relevant to our UC model (columns 0-25, i.e. first 26 columns)
gen_info = gen_info.iloc[:, :26]

# Outer join with fuel costs
gen_df = gen_info.merge(fuels, on='fuel', how='outer')
gen_df.rename(columns={'cost_per_mmbtu': 'fuel_cost'}, inplace=True)
gen_df['fuel_cost'] = gen_df['fuel_cost'].fillna(0)

# Create "is_variable" column
gen_df['is_variable'] = gen_df['resource'].isin(
    ['onshore_wind_turbine', 'small_hydroelectric', 'solar_photovoltaic']
)

# Create full name of generator (including geographic location and cluster number)
gen_df['gen_full'] = (gen_df['region'] + '_' + gen_df['resource'] + '_' + gen_df['cluster'].astype(str)).str.lower()

# Remove generators with no capacity
gen_df = gen_df[gen_df['existing_cap_mw'] > 0].reset_index(drop=True)

gen_df.head()

**Modify load and variable generation dataframes**

In [ ]:
# 1. Convert from GMT to GMT-8
gen_variable['hour'] = (gen_variable['hour'] - 9) % 8760 + 1
gen_variable = gen_variable.sort_values('hour').reset_index(drop=True)
loads['hour'] = (loads['hour'] - 9) % 8760 + 1
loads = loads.sort_values('hour').reset_index(drop=True)

# 2. Convert from "wide" to "long" format using pd.melt
gen_variable_long = pd.melt(
    gen_variable,
    id_vars=['hour'],
    var_name='gen_full',
    value_name='cf'
)

gen_variable_long.head()

### 2. Create solver functions

As a utility function, since we do this a lot, we'll create a function to convert Pyomo variable outputs with two indexes to DataFrames.

In [ ]:
def value_to_df_2dim(var, index1, index2, val_col='gen'):
    """
    Convert a Pyomo variable indexed by (index1, index2) to a pandas DataFrame.
    
    Parameters:
        var      -- Pyomo Var object indexed by two sets
        index1   -- list of first-dimension index values (e.g., generator r_ids)
        index2   -- list of second-dimension index values (e.g., hours)
        val_col  -- name for the value column (default: 'gen')
    
    Returns:
        DataFrame with columns [r_id, hour, val_col]
    """
    records = []
    for i in index1:
        for t in index2:
            records.append({
                'r_id': i,
                'hour': int(t),
                val_col: pyo.value(var[i, t])
            })
    return pd.DataFrame(records)

Then we'll create a function to create and solve a simple unit commitment problem with specified input data.

In [ ]:
def unit_commitment_simple(gen_df, loads, gen_variable, mip_gap):
    """
    Solve simple unit commitment problem (commitment equations).
    
    Parameters:
        gen_df       -- DataFrame with generator info
        loads        -- DataFrame with load by time
        gen_variable -- DataFrame with capacity factors of variable generators (long format)
        mip_gap      -- desired relative MIP gap
    
    Returns:
        tuple of (gen, commit, curtail, cost, status)
    """
    model = pyo.ConcreteModel('UC_Simple')

    # ----- Define sets based on data -----
    # Thermal resources (up_time > 0)
    G_thermal = list(gen_df.loc[gen_df['up_time'] > 0, 'r_id'])
    # Non-thermal resources (up_time == 0)
    G_nonthermal = list(gen_df.loc[gen_df['up_time'] == 0, 'r_id'])
    # Variable renewable resources
    G_var = list(gen_df.loc[gen_df['is_variable'] == True, 'r_id'])
    # Non-variable (dispatchable) resources
    G_nonvar = list(gen_df.loc[gen_df['is_variable'] == False, 'r_id'])
    # Non-variable and non-thermal resources
    G_nt_nonvar = list(set(G_nonvar) & set(G_nonthermal))
    # All generators
    G = list(gen_df['r_id'])
    # All time periods
    T = list(loads['hour'])
    # Reduced time periods (without last one)
    T_red = T[:-1]

    # Generator capacity factor time series for variable generators
    gen_var_cf = gen_variable.merge(
        gen_df.loc[gen_df['is_variable'] == True, ['r_id', 'gen_full', 'existing_cap_mw']],
        on='gen_full',
        how='inner'
    )

    # Build lookup dictionaries for generator parameters (for speed)
    gen_lookup = gen_df.set_index('r_id')

    # ----- Pyomo Sets -----
    model.G = pyo.Set(initialize=G)
    model.G_thermal = pyo.Set(initialize=G_thermal)
    model.T = pyo.Set(initialize=T)

    # ----- Decision Variables -----
    model.GEN = pyo.Var(G, T, domain=pyo.NonNegativeReals)
    model.COMMIT = pyo.Var(G_thermal, T, domain=pyo.Binary)
    model.START = pyo.Var(G_thermal, T, domain=pyo.Binary)
    model.SHUT = pyo.Var(G_thermal, T, domain=pyo.Binary)

    # ----- Objective Function -----
    def obj_rule(m):
        # Variable cost for non-variable generators
        cost_nonvar = sum(
            (gen_lookup.at[i, 'heat_rate_mmbtu_per_mwh'] * gen_lookup.at[i, 'fuel_cost'] +
             gen_lookup.at[i, 'var_om_cost_per_mwh']) * m.GEN[i, t]
            for i in G_nonvar for t in T
        )
        # Variable cost for variable generators
        cost_var = sum(
            gen_lookup.at[i, 'var_om_cost_per_mwh'] * m.GEN[i, t]
            for i in G_var for t in T
        )
        # Startup cost for thermal generators
        cost_start = sum(
            gen_lookup.at[i, 'start_cost_per_mw'] *
            gen_lookup.at[i, 'existing_cap_mw'] *
            m.START[i, t]
            for i in G_thermal for t in T
        )
        return cost_nonvar + cost_var + cost_start

    model.objective = pyo.Objective(rule=obj_rule, sense=pyo.minimize)

    # ----- Constraints -----

    # Demand balance
    demand_dict = dict(zip(loads['hour'], loads['demand']))

    def demand_rule(m, t):
        return sum(m.GEN[i, t] for i in G) == demand_dict[t]
    model.cDemand = pyo.Constraint(T, rule=demand_rule)

    # Capacity constraints for thermal generators (min)
    def cap_thermal_min_rule(m, i, t):
        return m.GEN[i, t] >= (m.COMMIT[i, t] *
                               gen_lookup.at[i, 'existing_cap_mw'] *
                               gen_lookup.at[i, 'min_power'])
    model.Cap_thermal_min = pyo.Constraint(G_thermal, T, rule=cap_thermal_min_rule)

    # Capacity constraints for thermal generators (max)
    def cap_thermal_max_rule(m, i, t):
        return m.GEN[i, t] <= m.COMMIT[i, t] * gen_lookup.at[i, 'existing_cap_mw']
    model.Cap_thermal_max = pyo.Constraint(G_thermal, T, rule=cap_thermal_max_rule)

    # Capacity constraints for non-thermal, non-variable generators
    def cap_nt_nonvar_rule(m, i, t):
        return m.GEN[i, t] <= gen_lookup.at[i, 'existing_cap_mw']
    model.Cap_nt_nonvar = pyo.Constraint(G_nt_nonvar, T, rule=cap_nt_nonvar_rule)

    # Capacity constraints for variable generation (using cf)
    model.Cap_var = pyo.ConstraintList()
    for _, row in gen_var_cf.iterrows():
        model.Cap_var.add(
            model.GEN[row['r_id'], row['hour']] <= row['cf'] * row['existing_cap_mw']
        )

    # Unit commitment constraints
    T_set = set(T)

    # 1. Minimum up time
    def startup_rule(m, i, t):
        up_time = int(gen_lookup.at[i, 'up_time'])
        relevant_t = [tt for tt in range(t - up_time, t + 1) if tt in T_set]
        return m.COMMIT[i, t] >= sum(m.START[i, tt] for tt in relevant_t)
    model.Startup = pyo.Constraint(G_thermal, T, rule=startup_rule)

    # 2. Minimum down time
    def shutdown_rule(m, i, t):
        down_time = int(gen_lookup.at[i, 'down_time'])
        relevant_t = [tt for tt in range(t - down_time, t + 1) if tt in T_set]
        return 1 - m.COMMIT[i, t] >= sum(m.SHUT[i, tt] for tt in relevant_t)
    model.Shutdown = pyo.Constraint(G_thermal, T, rule=shutdown_rule)

    # 3. Commitment status linking
    def commitment_status_rule(m, i, t):
        t_next = t + 1
        return m.COMMIT[i, t_next] - m.COMMIT[i, t] == m.START[i, t_next] - m.SHUT[i, t_next]
    model.CommitmentStatus = pyo.Constraint(G_thermal, T_red, rule=commitment_status_rule)

    # ----- Solve -----
    solver = pyo.SolverFactory('appsi_highs')
    solver.options['mip_rel_gap'] = mip_gap
    results = solver.solve(model, tee=True)

    # ----- Extract results -----
    # Generation
    gen = value_to_df_2dim(model.GEN, G, T, 'gen')

    # Commitment status
    commit = value_to_df_2dim(model.COMMIT, G_thermal, T, 'gen')

    # Curtailment
    curtail = gen_var_cf.merge(gen, on=['r_id', 'hour'], how='inner')
    curtail['curt'] = curtail['cf'] * curtail['existing_cap_mw'] - curtail['gen']

    cost = pyo.value(model.objective)
    status = results.solver.termination_condition

    return gen, commit, curtail, cost, status

### 3. Solve a day's unit commitment

In [ ]:
# A spring day
n = 100
T_period = list(range(n * 24 + 1, (n + 1) * 24 + 1))  # hours 2401-2424

# High solar case: 3,500 MW
gen_df_sens = gen_df.copy()
gen_df_sens.loc[gen_df_sens['resource'] == 'solar_photovoltaic', 'existing_cap_mw'] = 3500

loads_multi = loads[loads['hour'].isin(T_period)].reset_index(drop=True)
gen_variable_multi = gen_variable_long[gen_variable_long['hour'].isin(T_period)].reset_index(drop=True)

print(f"Time period: hours {T_period[0]} to {T_period[-1]}")
print(f"Number of generators: {len(gen_df_sens)}")
print(f"Number of hours: {len(T_period)}")

In [ ]:
# Note we reduce the MIP gap tolerance threshold here to increase tractability
# Here we set it to a 1% gap (mip_gap=0.01), meaning that we will terminate once we have
# a feasible integer solution guaranteed to be within 1% of the objective
# function value of the optimal solution (e.g. the upper and lower bound are within 1% of
# each other as HiGHS traverses the branch and bound tree).
# HiGHS's default MIP gap is 0.0001 (0.01%), which can take a longer time for
# any complex problem. So it is important to set this to a realistic value.

sol_gen_simple, sol_commit_simple, sol_curtail_simple, cost_simple, status_simple = \
    unit_commitment_simple(gen_df_sens, loads_multi, gen_variable_multi, 0.01)

print(f"\nStatus: {status_simple}")
print(f"Total cost: ${cost_simple:,.2f}")

In [ ]:
# Add in BTM solar and curtailment and plot results

def plot_generation(sol_gen_raw, sol_curtail, gen_df, gen_variable_multi, T_period, title='Generation Dispatch'):
    """
    Post-process and plot stacked area chart of generation dispatch.
    """
    # Join with resource names
    sol_gen = sol_gen_raw.merge(gen_df[['r_id', 'resource']], on='r_id', how='inner')
    sol_gen = sol_gen.groupby(['resource', 'hour'], as_index=False)['gen'].sum()

    # Rename generators (for plotting purposes -- underscore prefix for ordering)
    rename_map = {
        'solar_photovoltaic': '_solar_photovoltaic',
        'onshore_wind_turbine': '_onshore_wind_turbine',
        'small_hydroelectric': '_small_hydroelectric'
    }
    sol_gen['resource'] = sol_gen['resource'].replace(rename_map)

    # BTM solar (600 MW * cf)
    solar_cf = gen_variable_multi[
        gen_variable_multi['gen_full'] == 'wec_sdge_solar_photovoltaic_1.0'
    ][['hour', 'cf']].copy()
    btm = pd.DataFrame({
        'resource': '_solar_photovoltaic_btm',
        'hour': solar_cf['hour'].values,
        'gen': solar_cf['cf'].values * 600
    })
    sol_gen = pd.concat([sol_gen, btm], ignore_index=True)

    # Curtailment
    curtail = sol_curtail.groupby('hour', as_index=False)['curt'].sum()
    curtail['resource'] = '_curtailment'
    curtail.rename(columns={'curt': 'gen'}, inplace=True)
    sol_gen = pd.concat([sol_gen, curtail[['resource', 'hour', 'gen']]], ignore_index=True)

    # Rescale hours (relative to start of period)
    sol_gen['hour'] = sol_gen['hour'] - T_period[0]

    # Sort by resource name for consistent stacking
    sol_gen = sol_gen.sort_values(['resource', 'hour']).reset_index(drop=True)

    # Define a color map similar to category10
    resources = sorted(sol_gen['resource'].unique())
    category10 = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd',
                  '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf']
    color_map = {r: category10[i % len(category10)] for i, r in enumerate(resources)}

    fig = px.area(
        sol_gen, x='hour', y='gen', color='resource',
        title=title,
        labels={'hour': 'Hour', 'gen': 'Generation (MW)', 'resource': 'Resource'},
        color_discrete_map=color_map
    )
    fig.update_layout(xaxis_title='Hour', yaxis_title='Generation (MW)')
    fig.show()

    return sol_gen

sol_gen_plot = plot_generation(
    sol_gen_simple, sol_curtail_simple, gen_df, gen_variable_multi, T_period,
    title='Simple UC - Generation Dispatch (Spring Day, 3500 MW Solar)'
)

Notice that the combined cycle plants flatten out during the day, but don't shut down due to the need for ramping capabilities in the late afternoon/evening, when solar output falls off and evening demand remains strong. This leads to some curtailment of solar from 11:00 - 16:00, which was not present in the Economic Dispatch model. All curtailment is lumped together as the model does not accurately distinguish between whether wind, solar or hydro is curtailed, which in practice is up to the system operator and depends on a variety of factors such as location, interconnection voltage, etc.

We can examine the commitment status of various units by examining the results in `sol_commit_simple`.

In [ ]:
def plot_commitment(sol_commit_raw, gen_df, T_period, title='Commitment Status'):
    """
    Post-process and plot stacked area chart of commitment status.
    """
    sol_commit = sol_commit_raw.merge(gen_df[['r_id', 'resource']], on='r_id', how='inner')
    sol_commit = sol_commit.groupby(['resource', 'hour'], as_index=False)['gen'].sum()
    sol_commit['hour'] = sol_commit['hour'] - T_period[0]

    resources = sorted(sol_commit['resource'].unique())
    category10 = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd',
                  '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf']
    color_map = {r: category10[i % len(category10)] for i, r in enumerate(resources)}

    fig = px.area(
        sol_commit, x='hour', y='gen', color='resource',
        title=title,
        labels={'hour': 'Hour', 'gen': 'Committed Units', 'resource': 'Resource'},
        color_discrete_map=color_map
    )
    fig.update_layout(xaxis_title='Hour', yaxis_title='Committed Units')
    fig.show()

plot_commitment(
    sol_commit_simple, gen_df, T_period,
    title='Simple UC - Commitment Status (Spring Day, 3500 MW Solar)'
)

Note that units shutdown during the solar period and startup for the evening peak. However, due to the commitment constraints, not all natural gas plants can be decommitted, resulting in the observed curtailment. A large number of combustion turbines turn on to meet the evening peak.

## Moderate complexity unit commitment

We expand the above unit commitment with ramp equations. These must be modified to prevent ramp violations during the startup process. In order to accommodate this, we introduce an **auxiliary variable** $GENAUX_{g,t}$ defined as the generation above the minimum output (if committed). $GENAUX_{g,t} = GEN_{g,t} = 0$ if the unit is not committed. Auxiliary variables are for convenience (we could always write the whole problem in terms of the original decision variables), but they help de-clutter the model and in general won't add too much computational penalty. The following constraints are added:

$$
\begin{align}
 & GENAUX_{g,t} = GEN_{g,t} - Pmin_{g,t}COMMIT_{g,t} & \forall \quad g \in G_{thermal} , t \in T \\
 & GENAUX_{g,t+1} - GENAUX_{g,t} \leq RampUp_{g} & \forall \quad g \in G_{thermal} , t = 1..T-1 \\
  & GENAUX_{g,t} - GENAUX_{g,t+1} \leq RampDn_{g} & \forall \quad g \in G_{thermal} , t = 1..T-1 \\
 & GEN_{g,t+1} - GEN_{g,t} \leq RampUp_{g} & \forall \quad g \notin G_{thermal} , t = 1..T-1 \\
  & GEN_{g,t} - GEN_{g,t+1} \leq RampDn_{g} & \forall \quad g \notin G_{thermal} , t = 1..T-1  
\end{align}
$$

The creation of this auxiliary variable $GENAUX$ helps us avoid violating ramping constraints when we start up our units and their generation immediately jumps up to their minium output level or during shut-down when a unit drops from minimum output (or above) to zero.

Note you may encounter alternative formulations in the literature to address this start-up and shut-down that modify the right-hand side of the traditional ramping constraint on total generation to account for the additional step change in output during start-up or shut-down periods. The above formulation is conceptually simple and works well.

### 3. Create solver function

(We reuse steps 1 and 2 above to load packages and data.)

In [ ]:
def unit_commitment_ramp(gen_df, loads, gen_variable, mip_gap):
    """
    Solve moderate complexity unit commitment problem (commitment and ramp equations).
    
    Parameters:
        gen_df       -- DataFrame with generator info
        loads        -- DataFrame with load by time
        gen_variable -- DataFrame with capacity factors of variable generators (long format)
        mip_gap      -- desired relative MIP gap
    
    Returns:
        tuple of (gen, commit, curtail, cost, status)
    """
    model = pyo.ConcreteModel('UC_Ramp')

    # ----- Define sets based on data -----
    G_thermal = list(gen_df.loc[gen_df['up_time'] > 0, 'r_id'])
    G_nonthermal = list(gen_df.loc[gen_df['up_time'] == 0, 'r_id'])
    G_var = list(gen_df.loc[gen_df['is_variable'] == True, 'r_id'])
    G_nonvar = list(gen_df.loc[gen_df['is_variable'] == False, 'r_id'])
    G_nt_nonvar = list(set(G_nonvar) & set(G_nonthermal))
    G = list(gen_df['r_id'])
    T = list(loads['hour'])
    T_red = T[:-1]

    gen_var_cf = gen_variable.merge(
        gen_df.loc[gen_df['is_variable'] == True, ['r_id', 'gen_full', 'existing_cap_mw']],
        on='gen_full', how='inner'
    )

    gen_lookup = gen_df.set_index('r_id')

    # ----- Pyomo Sets -----
    model.G = pyo.Set(initialize=G)
    model.G_thermal = pyo.Set(initialize=G_thermal)
    model.T = pyo.Set(initialize=T)

    # ----- Decision Variables -----
    model.GEN = pyo.Var(G, T, domain=pyo.NonNegativeReals)
    model.GENAUX = pyo.Var(G_thermal, T, domain=pyo.NonNegativeReals)
    model.COMMIT = pyo.Var(G_thermal, T, domain=pyo.Binary)
    model.START = pyo.Var(G_thermal, T, domain=pyo.Binary)
    model.SHUT = pyo.Var(G_thermal, T, domain=pyo.Binary)

    # ----- Objective Function -----
    def obj_rule(m):
        cost_nonvar = sum(
            (gen_lookup.at[i, 'heat_rate_mmbtu_per_mwh'] * gen_lookup.at[i, 'fuel_cost'] +
             gen_lookup.at[i, 'var_om_cost_per_mwh']) * m.GEN[i, t]
            for i in G_nonvar for t in T
        )
        cost_var = sum(
            gen_lookup.at[i, 'var_om_cost_per_mwh'] * m.GEN[i, t]
            for i in G_var for t in T
        )
        cost_start = sum(
            gen_lookup.at[i, 'start_cost_per_mw'] *
            gen_lookup.at[i, 'existing_cap_mw'] *
            m.START[i, t]
            for i in G_thermal for t in T
        )
        return cost_nonvar + cost_var + cost_start

    model.objective = pyo.Objective(rule=obj_rule, sense=pyo.minimize)

    # ----- Constraints -----

    # Demand balance
    demand_dict = dict(zip(loads['hour'], loads['demand']))

    def demand_rule(m, t):
        return sum(m.GEN[i, t] for i in G) == demand_dict[t]
    model.cDemand = pyo.Constraint(T, rule=demand_rule)

    # Capacity constraints (thermal)
    def cap_thermal_min_rule(m, i, t):
        return m.GEN[i, t] >= (m.COMMIT[i, t] *
                               gen_lookup.at[i, 'existing_cap_mw'] *
                               gen_lookup.at[i, 'min_power'])
    model.Cap_thermal_min = pyo.Constraint(G_thermal, T, rule=cap_thermal_min_rule)

    def cap_thermal_max_rule(m, i, t):
        return m.GEN[i, t] <= m.COMMIT[i, t] * gen_lookup.at[i, 'existing_cap_mw']
    model.Cap_thermal_max = pyo.Constraint(G_thermal, T, rule=cap_thermal_max_rule)

    # Capacity constraints (non-thermal, non-variable)
    def cap_nt_nonvar_rule(m, i, t):
        return m.GEN[i, t] <= gen_lookup.at[i, 'existing_cap_mw']
    model.Cap_nt_nonvar = pyo.Constraint(G_nt_nonvar, T, rule=cap_nt_nonvar_rule)

    # Capacity constraints (variable generation)
    model.Cap_var = pyo.ConstraintList()
    for _, row in gen_var_cf.iterrows():
        model.Cap_var.add(
            model.GEN[row['r_id'], row['hour']] <= row['cf'] * row['existing_cap_mw']
        )

    # Unit commitment constraints
    T_set = set(T)

    def startup_rule(m, i, t):
        up_time = int(gen_lookup.at[i, 'up_time'])
        relevant_t = [tt for tt in range(t - up_time, t + 1) if tt in T_set]
        return m.COMMIT[i, t] >= sum(m.START[i, tt] for tt in relevant_t)
    model.Startup = pyo.Constraint(G_thermal, T, rule=startup_rule)

    def shutdown_rule(m, i, t):
        down_time = int(gen_lookup.at[i, 'down_time'])
        relevant_t = [tt for tt in range(t - down_time, t + 1) if tt in T_set]
        return 1 - m.COMMIT[i, t] >= sum(m.SHUT[i, tt] for tt in relevant_t)
    model.Shutdown = pyo.Constraint(G_thermal, T, rule=shutdown_rule)

    def commitment_status_rule(m, i, t):
        t_next = t + 1
        return m.COMMIT[i, t_next] - m.COMMIT[i, t] == m.START[i, t_next] - m.SHUT[i, t_next]
    model.CommitmentStatus = pyo.Constraint(G_thermal, T_red, rule=commitment_status_rule)

    # Auxiliary variable constraint: GENAUX = GEN - COMMIT * Pmin * Cap
    def auxgen_rule(m, i, t):
        return m.GENAUX[i, t] == (m.GEN[i, t] -
                                  m.COMMIT[i, t] *
                                  gen_lookup.at[i, 'existing_cap_mw'] *
                                  gen_lookup.at[i, 'min_power'])
    model.AuxGen = pyo.Constraint(G_thermal, T, rule=auxgen_rule)

    # Ramp constraints for thermal generators (on GENAUX)
    def rampup_thermal_rule(m, i, t):
        t_next = t + 1
        return (m.GENAUX[i, t_next] - m.GENAUX[i, t] <=
                gen_lookup.at[i, 'existing_cap_mw'] *
                gen_lookup.at[i, 'ramp_up_percentage'])
    model.RampUp_thermal = pyo.Constraint(G_thermal, T_red, rule=rampup_thermal_rule)

    def rampdn_thermal_rule(m, i, t):
        t_next = t + 1
        return (m.GENAUX[i, t] - m.GENAUX[i, t_next] <=
                gen_lookup.at[i, 'existing_cap_mw'] *
                gen_lookup.at[i, 'ramp_dn_percentage'])
    model.RampDn_thermal = pyo.Constraint(G_thermal, T_red, rule=rampdn_thermal_rule)

    # Ramp constraints for non-thermal generators (on GEN)
    def rampup_nonthermal_rule(m, i, t):
        t_next = t + 1
        return (m.GEN[i, t_next] - m.GEN[i, t] <=
                gen_lookup.at[i, 'existing_cap_mw'] *
                gen_lookup.at[i, 'ramp_up_percentage'])
    model.RampUp_nonthermal = pyo.Constraint(G_nonthermal, T_red, rule=rampup_nonthermal_rule)

    # NOTE: In the Julia code, RampDn applies to ALL generators (G), not just non-thermal
    def rampdn_rule(m, i, t):
        t_next = t + 1
        return (m.GEN[i, t] - m.GEN[i, t_next] <=
                gen_lookup.at[i, 'existing_cap_mw'] *
                gen_lookup.at[i, 'ramp_dn_percentage'])
    model.RampDn = pyo.Constraint(G, T_red, rule=rampdn_rule)

    # ----- Solve -----
    solver = pyo.SolverFactory('appsi_highs')
    solver.options['mip_rel_gap'] = mip_gap
    results = solver.solve(model, tee=True)

    # ----- Extract results -----
    gen = value_to_df_2dim(model.GEN, G, T, 'gen')
    commit = value_to_df_2dim(model.COMMIT, G_thermal, T, 'gen')

    curtail = gen_var_cf.merge(gen, on=['r_id', 'hour'], how='inner')
    curtail['curt'] = curtail['cf'] * curtail['existing_cap_mw'] - curtail['gen']

    cost = pyo.value(model.objective)
    status = results.solver.termination_condition

    return gen, commit, curtail, cost, status

### 4. Solve a day's unit commitment

In [ ]:
sol_gen_ramp, sol_commit_ramp, sol_curtail_ramp, cost_ramp, status_ramp = \
    unit_commitment_ramp(gen_df_sens, loads_multi, gen_variable_multi, 0.01)

print(f"\nStatus: {status_ramp}")
print(f"Total cost: ${cost_ramp:,.2f}")

In [ ]:
# Plot generation dispatch
sol_gen_ramp_plot = plot_generation(
    sol_gen_ramp, sol_curtail_ramp, gen_df, gen_variable_multi, T_period,
    title='UC with Ramp - Generation Dispatch (Spring Day, 3500 MW Solar)'
)

In contrast to when we added ramp constraints to the Economic Dispatch problem, adding ramp constraints to our Unit Commitment formulation does not change the solution as much (at least in this case). Much of the inflexibility here arises from the unit commitment constraints, and once committed, there is ample ramping capability. We do however get relatively more combustion turbines running to meet the evening ramp, given their faster ramp rates.

In [ ]:
# Plot commitment status
plot_commitment(
    sol_commit_ramp, gen_df, T_period,
    title='UC with Ramp - Commitment Status (Spring Day, 3500 MW Solar)'
)

A case involving more large coal or nuclear units with slower ramping limits than gas turbines or combined cycle plants might find ramp constraints more limiting. As in many cases, the impact of constraints is case dependent, and modelers must exercise some judicious application of domain knowledge (and experimentation with alternative formulations) to decide how complex your model should be to reflect important *binding* constraints that actually shape your outcomes of interest. Where constraints are non-binding or second order, it may be more practical to omit them entirely, reducing the computational intensity (solve time) of your model...

## Unit commitment with ramping and reserves

We now add spinning reserve requirements to the model. Spinning reserves refer to generation capacity that is online and able to generate (e.g., within 10-30 minutes) if needed by the system operator. (Note: in Europe, these are referred to as 'replacement reserves' or formerly 'tertiary reserves.') The SO will establish reserve requirements to maintain sufficient capacity to respond to demand or supply forecast errors or in case of "contingencies," such as the unplanned and sudden loss of a generating station or a transmission line. The SO then typically operates a reserve market to competitively procure this available capacity. Most SO's actually define several classes of reserve products defined by their response time and period over which they may be activated.

Here we will focus on spinning reserve requirements and establish a simple set of reserve requirements for reserves up (ability to quickly increase output) and reserves down (ability to quickly reduce output):

$$
\begin{align}
 & ResReqUp_t = 300 MW + 5\% \times Demand_t  &\forall \quad t \in T \\
 & ResReqDn_t = 5\% \times Demand_t & \forall \quad t \in T
\end{align}
$$

Here, $300 MW$ is our contigency reserves, meant to ensure we have sufficient upwards ramping capability online and available to cover the unexpected loss of 300 MW worth of generation.

The 5% of demand term in the constraints above is meant to provide sufficient reserves in either upwards or downwards direction to cover errors in the demand forecast. For simplicity, we ignore solar PV and wind production in our reserve calculation, though in practice, we would want to consider solar and wind forecast errors in calculating up reserve requirements as well.

In our simple system, only thermal generators provide reserves. (This can be easily adjusted by changing the sets and defining the potential reserve contributions for different sets of resources differently, such as storage or hydro resources). The contribution of each generators to meeting reserves and the overall reserve constraint are thus given by:

$$
\begin{align}
 & RESUP_{g,t} \leq Pmax_{g,t}COMMIT_{g,t} - GEN_{g,t} & \forall \quad g \in G_{thermal} , t \in T \\
 & RESDN_{g,t} \leq GEN_{g,t} - Pmin_{g,t}COMMIT_{g,t} & \forall \quad g \in G_{thermal} , t \in T \\
 & RESUP_{g,t} \leq RampUp_{g} & \forall \quad g \in G_{thermal}, t \in T\\
 & RESDN_{g,t} \leq RampDn_{g} & \forall \quad g \in G_{thermal}, t \in T \\
 & \sum_{g \in G_{thermal}} RESUP_{g,t} \geq ResReqUp_t & \forall \quad t \in T \\
 & \sum_{g \in G_{thermal}} RESDN_{g,t} \geq ResReqDn_t & \forall \quad t \in T
\end{align}
$$

We have added two new **decision variables**:

- $RESUP_{g,t}$, up-reserve capacity (in MW) of generator $g$ at time $t$
- $RESDN_{g,t}$, down-reserve capacity (in MW) of generator $g$ at time $t$

Note that in this case, we constrain the reserve contribution for each thermal generator to be the same as their hourly ramp rates, $RampUp_{g}, RampDown_{g}$. In practice, reserve products typically require a faster response time, on the order of 10-30 minutes for "tertiary" reserves (aka spinning reserves or contingency reserves), and 5-15 minutes for "secondary" reserves (regulation reserves). Thus, depending on the reserve requirements being modeled, one might specify a distinct maximum reserve contribution for each unit that reflects their ramping capabilities over shorter time periods.

<img src="https://github.com/Power-Systems-Optimization-Course/power-systems-optimization/blob/master/Notebooks/img/reserves.png?raw=1" style="width: 450px; height: auto" align="left">

<img src="https://github.com/Power-Systems-Optimization-Course/power-systems-optimization/blob/master/Notebooks/img/reserves_taxonomy.png?raw=1" style="width: 450px; height: auto" align="left">

### 3. Create solver function

(We reuse steps 1 and 2 above to load packages and data.)

In [ ]:
def unit_commitment_full(gen_df, loads, gen_variable, mip_gap):
    """
    Solve full unit commitment problem (commitment, ramp, reserve equations).
    
    Parameters:
        gen_df       -- DataFrame with generator info
        loads        -- DataFrame with load by time
        gen_variable -- DataFrame with capacity factors of variable generators (long format)
        mip_gap      -- desired relative MIP gap
    
    Returns:
        tuple of (gen, commit, curtail, cost, status)
    """
    model = pyo.ConcreteModel('UC_Full')

    # ----- Define sets based on data -----
    G_thermal = list(gen_df.loc[gen_df['up_time'] > 0, 'r_id'])
    G_nonthermal = list(gen_df.loc[gen_df['up_time'] == 0, 'r_id'])
    G_var = list(gen_df.loc[gen_df['is_variable'] == True, 'r_id'])
    G_nonvar = list(gen_df.loc[gen_df['is_variable'] == False, 'r_id'])
    G_nt_nonvar = list(set(G_nonvar) & set(G_nonthermal))
    G = list(gen_df['r_id'])
    T = list(loads['hour'])
    T_red = T[:-1]

    gen_var_cf = gen_variable.merge(
        gen_df.loc[gen_df['is_variable'] == True, ['r_id', 'gen_full', 'existing_cap_mw']],
        on='gen_full', how='inner'
    )
    gen_var_cf['max_gen'] = gen_var_cf['cf'] * gen_var_cf['existing_cap_mw']

    gen_lookup = gen_df.set_index('r_id')

    # Compute reserve requirements
    demand_dict = dict(zip(loads['hour'], loads['demand']))
    ResReqUp = {t: 300 + 0.05 * demand_dict[t] for t in T}
    ResReqDn = {t: 0.05 * demand_dict[t] for t in T}

    # ----- Pyomo Sets -----
    model.G = pyo.Set(initialize=G)
    model.G_thermal = pyo.Set(initialize=G_thermal)
    model.T = pyo.Set(initialize=T)

    # ----- Decision Variables -----
    model.GEN = pyo.Var(G, T, domain=pyo.NonNegativeReals)
    model.GENAUX = pyo.Var(G_thermal, T, domain=pyo.NonNegativeReals)
    model.COMMIT = pyo.Var(G_thermal, T, domain=pyo.Binary)
    model.START = pyo.Var(G_thermal, T, domain=pyo.Binary)
    model.SHUT = pyo.Var(G_thermal, T, domain=pyo.Binary)
    model.RESUP = pyo.Var(G_thermal, T, domain=pyo.NonNegativeReals)
    model.RESDN = pyo.Var(G_thermal, T, domain=pyo.NonNegativeReals)

    # ----- Objective Function -----
    def obj_rule(m):
        cost_nonvar = sum(
            (gen_lookup.at[i, 'heat_rate_mmbtu_per_mwh'] * gen_lookup.at[i, 'fuel_cost'] +
             gen_lookup.at[i, 'var_om_cost_per_mwh']) * m.GEN[i, t]
            for i in G_nonvar for t in T
        )
        cost_var = sum(
            gen_lookup.at[i, 'var_om_cost_per_mwh'] * m.GEN[i, t]
            for i in G_var for t in T
        )
        cost_start = sum(
            gen_lookup.at[i, 'start_cost_per_mw'] *
            gen_lookup.at[i, 'existing_cap_mw'] *
            m.START[i, t]
            for i in G_thermal for t in T
        )
        return cost_nonvar + cost_var + cost_start

    model.objective = pyo.Objective(rule=obj_rule, sense=pyo.minimize)

    # ----- Constraints -----

    # Demand balance
    def demand_rule(m, t):
        return sum(m.GEN[i, t] for i in G) == demand_dict[t]
    model.cDemand = pyo.Constraint(T, rule=demand_rule)

    # Capacity constraints (thermal)
    def cap_thermal_min_rule(m, i, t):
        return m.GEN[i, t] >= (m.COMMIT[i, t] *
                               gen_lookup.at[i, 'existing_cap_mw'] *
                               gen_lookup.at[i, 'min_power'])
    model.Cap_thermal_min = pyo.Constraint(G_thermal, T, rule=cap_thermal_min_rule)

    def cap_thermal_max_rule(m, i, t):
        return m.GEN[i, t] <= m.COMMIT[i, t] * gen_lookup.at[i, 'existing_cap_mw']
    model.Cap_thermal_max = pyo.Constraint(G_thermal, T, rule=cap_thermal_max_rule)

    # Capacity constraints (non-thermal, non-variable)
    def cap_nt_nonvar_rule(m, i, t):
        return m.GEN[i, t] <= gen_lookup.at[i, 'existing_cap_mw']
    model.Cap_nt_nonvar = pyo.Constraint(G_nt_nonvar, T, rule=cap_nt_nonvar_rule)

    # Capacity constraints (variable generation)
    model.Cap_var = pyo.ConstraintList()
    for _, row in gen_var_cf.iterrows():
        model.Cap_var.add(
            model.GEN[row['r_id'], row['hour']] <= row['max_gen']
        )

    # Unit commitment constraints
    T_set = set(T)

    def startup_rule(m, i, t):
        up_time = int(gen_lookup.at[i, 'up_time'])
        relevant_t = [tt for tt in range(t - up_time, t + 1) if tt in T_set]
        return m.COMMIT[i, t] >= sum(m.START[i, tt] for tt in relevant_t)
    model.Startup = pyo.Constraint(G_thermal, T, rule=startup_rule)

    def shutdown_rule(m, i, t):
        down_time = int(gen_lookup.at[i, 'down_time'])
        relevant_t = [tt for tt in range(t - down_time, t + 1) if tt in T_set]
        return 1 - m.COMMIT[i, t] >= sum(m.SHUT[i, tt] for tt in relevant_t)
    model.Shutdown = pyo.Constraint(G_thermal, T, rule=shutdown_rule)

    def commitment_status_rule(m, i, t):
        t_next = t + 1
        return m.COMMIT[i, t_next] - m.COMMIT[i, t] == m.START[i, t_next] - m.SHUT[i, t_next]
    model.CommitmentStatus = pyo.Constraint(G_thermal, T_red, rule=commitment_status_rule)

    # Auxiliary variable constraint
    def auxgen_rule(m, i, t):
        return m.GENAUX[i, t] == (m.GEN[i, t] -
                                  m.COMMIT[i, t] *
                                  gen_lookup.at[i, 'existing_cap_mw'] *
                                  gen_lookup.at[i, 'min_power'])
    model.AuxGen = pyo.Constraint(G_thermal, T, rule=auxgen_rule)

    # Ramp constraints for thermal generators (on GENAUX)
    def rampup_thermal_rule(m, i, t):
        t_next = t + 1
        return (m.GENAUX[i, t_next] - m.GENAUX[i, t] <=
                gen_lookup.at[i, 'existing_cap_mw'] *
                gen_lookup.at[i, 'ramp_up_percentage'])
    model.RampUp_thermal = pyo.Constraint(G_thermal, T_red, rule=rampup_thermal_rule)

    def rampdn_thermal_rule(m, i, t):
        t_next = t + 1
        return (m.GENAUX[i, t] - m.GENAUX[i, t_next] <=
                gen_lookup.at[i, 'existing_cap_mw'] *
                gen_lookup.at[i, 'ramp_dn_percentage'])
    model.RampDn_thermal = pyo.Constraint(G_thermal, T_red, rule=rampdn_thermal_rule)

    # Ramp constraints for non-thermal generators (on GEN)
    def rampup_nonthermal_rule(m, i, t):
        t_next = t + 1
        return (m.GEN[i, t_next] - m.GEN[i, t] <=
                gen_lookup.at[i, 'existing_cap_mw'] *
                gen_lookup.at[i, 'ramp_up_percentage'])
    model.RampUp_nonthermal = pyo.Constraint(G_nonthermal, T_red, rule=rampup_nonthermal_rule)

    # NOTE: In the Julia code, RampDn applies to ALL generators (G), not just non-thermal
    def rampdn_rule(m, i, t):
        t_next = t + 1
        return (m.GEN[i, t] - m.GEN[i, t_next] <=
                gen_lookup.at[i, 'existing_cap_mw'] *
                gen_lookup.at[i, 'ramp_dn_percentage'])
    model.RampDn = pyo.Constraint(G, T_red, rule=rampdn_rule)

    # ----- Reserve Constraints -----

    # (1) Reserves limited by committed capacity
    def resupcap_rule(m, i, t):
        return m.RESUP[i, t] <= (m.COMMIT[i, t] * gen_lookup.at[i, 'existing_cap_mw']
                                 - m.GEN[i, t])
    model.ResUpCap = pyo.Constraint(G_thermal, T, rule=resupcap_rule)

    def resdncap_rule(m, i, t):
        return m.RESDN[i, t] <= (m.GEN[i, t] -
                                 m.COMMIT[i, t] *
                                 gen_lookup.at[i, 'existing_cap_mw'] *
                                 gen_lookup.at[i, 'min_power'])
    model.ResDnCap = pyo.Constraint(G_thermal, T, rule=resdncap_rule)

    # (2) Reserves limited by ramp rates
    def resupramp_rule(m, i, t):
        return m.RESUP[i, t] <= (gen_lookup.at[i, 'existing_cap_mw'] *
                                 gen_lookup.at[i, 'ramp_up_percentage'])
    model.ResUpRamp = pyo.Constraint(G_thermal, T, rule=resupramp_rule)

    def resdnramp_rule(m, i, t):
        return m.RESDN[i, t] <= (gen_lookup.at[i, 'existing_cap_mw'] *
                                 gen_lookup.at[i, 'ramp_dn_percentage'])
    model.ResDnRamp = pyo.Constraint(G_thermal, T, rule=resdnramp_rule)

    # (3) Overall reserve requirements
    def resupreq_rule(m, t):
        return sum(m.RESUP[i, t] for i in G_thermal) >= ResReqUp[t]
    model.ResUpRequirement = pyo.Constraint(T, rule=resupreq_rule)

    def resdnreq_rule(m, t):
        return sum(m.RESDN[i, t] for i in G_thermal) >= ResReqDn[t]
    model.ResDnRequirement = pyo.Constraint(T, rule=resdnreq_rule)

    # ----- Solve -----
    solver = pyo.SolverFactory('appsi_highs')
    solver.options['mip_rel_gap'] = mip_gap
    results = solver.solve(model, tee=True)

    # ----- Extract results -----
    gen = value_to_df_2dim(model.GEN, G, T, 'gen')
    commit = value_to_df_2dim(model.COMMIT, G_thermal, T, 'gen')

    curtail = gen_var_cf.merge(gen, on=['r_id', 'hour'], how='inner')
    curtail['curt'] = curtail['cf'] * curtail['existing_cap_mw'] - curtail['gen']

    cost = pyo.value(model.objective)
    status = results.solver.termination_condition

    return gen, commit, curtail, cost, status

### 4. Solve the model and plot results

**Note: this might take a little while depending on your machine.**

In [ ]:
# To keep solve times to a few minutes, we increase the MIP gap tolerance
# in this more complex case to 2% (mip_gap=0.02). You can play with this setting to see
# how the MIP gap affects solve times on your computer.

sol_gen_full, sol_commit_full, sol_curtail_full, cost_full, status_full = \
    unit_commitment_full(gen_df_sens, loads_multi, gen_variable_multi, 0.02)

print(f"\nStatus: {status_full}")
print(f"Total cost: ${cost_full:,.2f}")

In [ ]:
# Plot generation dispatch
sol_gen_full_plot = plot_generation(
    sol_gen_full, sol_curtail_full, gen_df, gen_variable_multi, T_period,
    title='Full UC - Generation Dispatch (Spring Day, 3500 MW Solar)'
)

We can now see that the need to maintain reserve requirements leads to greater commitment of natural gas units throughout the day. The addition of CCGTs and GTs to meet reserve requirements leads to greater curtailment of wind and solar in the afternoon hours. We also see more CT units committed during the morning ramp as well as the afternoon hours to help provide reserves.

In [ ]:
# Plot commitment status
plot_commitment(
    sol_commit_full, gen_df, T_period,
    title='Full UC - Commitment Status (Spring Day, 3500 MW Solar)'
)

### Further resources

Knueven, B., Ostrowski, J., & Watson, J.-P. (2019). On Mixed Integer Programming Formulations for the Unit Commitment Problem. Optimization Online, 91. http://www.optimization-online.org/DB_FILE/2018/11/6930.pdf

Morales-Espana, G., Latorre, J. M., & Ramos, A. (2013). Tight and Compact MILP Formulation of Start-Up and Shut-Down Ramping in Unit Commitment. IEEE Transactions on Power Systems, 28(2), 1288-1296. https://doi.org/10.1109/TPWRS.2012.2222938

Morales-Espana, G., Ramirez-Elizondo, L., & Hobbs, B. F. (2017). Hidden power system inflexibilities imposed by traditional unit commitment formulations. Applied Energy, 191, 223-238. https://doi.org/10.1016/j.apenergy.2017.01.089

Ostrowski, J., Anjos, M. F., & Vannelli, A. (2012). Tight Mixed Integer Linear Programming Formulations for the Unit Commitment Problem. IEEE Transactions on Power Systems, 27(1), 39-46. https://doi.org/10.1109/TPWRS.2011.2162008